# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [9]:
from string import punctuation

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    
    reference_text = ''.join([ch for ch in reference_text.strip().lower() if ch not in punctuation])
    reference_words = reference_text.split()
    
    recognized_text = ''.join([ch for ch in recognized_text.strip().lower() if ch not in punctuation])
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                distance_matrix[i][j] = distance_matrix[i - 1][j - 1]
            else:
                distance_matrix[i][j] = 1 + min(distance_matrix[i][j - 1], distance_matrix[i - 1][j], distance_matrix[i - 1][j - 1])

    # Расчет WER  (в процентах)
    wer = distance_matrix[len(reference_words)][len(recognized_words)]/len(reference_words)
    return wer * 100

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [10]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [1]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [33]:
from collections import Counter
import numpy as np

def calculate_wer_with_alignment(reference_text: str, recognized_text: str): 
    # Перенесите сюда код из задания 1.a.
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    
    reference_text = ''.join([ch for ch in reference_text.strip().lower() if ch not in punctuation])
    reference_words = reference_text.split()
    
    recognized_text = ''.join([ch for ch in recognized_text.strip().lower() if ch not in punctuation])
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    path_matrix = [[list() for i in range((len(recognized_words) + 1))] for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
        if i > 0:
            path_matrix[i][0].append("D")

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j
        if j > 0:
            path_matrix[0][j].append("I")

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                distance_matrix[i][j] = distance_matrix[i - 1][j - 1]
                path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['C']
            else:
                choices = [distance_matrix[i][j - 1], distance_matrix[i - 1][j], distance_matrix[i - 1][j - 1]]
                arg_min = np.argmin(choices)
                min_value = choices[arg_min]
                
                match arg_min:
                    case 0:
                        path_matrix[i][j] = path_matrix[i][j - 1] + ['I']
                    case 1:
                        path_matrix[i][j] = path_matrix[i - 1][j] + ['D']
                    case 2:
                        path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['S']
                
                distance_matrix[i][j] = 1 + min_value

    # Расчет WER  (в процентах)
    wer = distance_matrix[-1][-1]/len(reference_words) * 100
    
    annot = path_matrix[-1][-1]
    ref_ali = []
    rec_ali = []
    
    ref_w_iter = iter(reference_words)
    rec_w_iter = iter(recognized_words)
    
    for symbol in annot:
        if symbol == 'D':
            ref_ali.append(next(ref_w_iter))
            rec_ali.append("***")
        elif symbol == 'I':
            ref_ali.append("***")
            rec_ali.append(next(rec_w_iter))
        elif symbol == 'C' or symbol == 'S':
            ref_ali.append(next(ref_w_iter))
            rec_ali.append(next(rec_w_iter))

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    #TODO
    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    
    ali = [ref_ali, rec_ali, annot]
    
    counts = Counter(annot)
    correct = counts['C']
    deletion = counts['D']
    insertion = counts['I']
    substitution = counts['S']
    
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [34]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [235]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    reference_text = ''.join([ch if ch not in punctuation else f" {ch} " for ch in reference_text.strip().lower()])
    reference_words = reference_text.split()
    
    recognized_text = ''.join([ch if ch not in punctuation else f" {ch} "  for ch in recognized_text.strip().lower()])
    recognized_words = recognized_text.split()
    
    # расстояние Левенштейна 
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    path_matrix = [[list() for i in range((len(recognized_words) + 1))] for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
        if i > 0: 
            if reference_words[i-1] not in punctuation:
                path_matrix[i][0].append("D")
            else:
                path_matrix[i][0].append("D_p")

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j
        if j > 0:
            if recognized_words[j-1] not in punctuation:
                path_matrix[0][j].append("I")
            else:
                path_matrix[0][j].append("I_p")

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                distance_matrix[i][j] = distance_matrix[i - 1][j - 1]
                if reference_words[i - 1] in punctuation or recognized_words[j - 1] in punctuation:
                    path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['C_p']
                else:
                    path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['C']
            else:
                choices = [distance_matrix[i - 1][j], distance_matrix[i][j - 1], distance_matrix[i - 1][j - 1]]
                
                if reference_words[i-1] in punctuation or recognized_words[j-1] in punctuation:
                    choices[-1] = float("inf")
                    
                if reference_words[i-1] in punctuation and recognized_words[j-1] in punctuation:
                    choices[-1] = distance_matrix[i - 1][j - 1]
                
                
                arg_min = np.argmin(choices)
                min_value = choices[arg_min]
                
                match arg_min:
                    case 1:
                        if recognized_words[j-1] in punctuation:
                            path_matrix[i][j] = path_matrix[i][j - 1] + ['I_p']
                        else:
                            path_matrix[i][j] = path_matrix[i][j - 1] + ['I']
                    case 0:
                        if reference_words[i-1] in punctuation:
                            path_matrix[i][j] = path_matrix[i - 1][j] + ['D_p']
                        else:
                            path_matrix[i][j] = path_matrix[i - 1][j] + ['D']
                    case 2:
                        if reference_words[i-1] in punctuation and recognized_words[j-1] in punctuation:
                            path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['S_p']
                        elif reference_words[i-1] not in punctuation and recognized_words[j-1] not in punctuation:
                            path_matrix[i][j] = path_matrix[i - 1][j - 1] + ['S']
                distance_matrix[i][j] = 1 + min_value
    # Расчет WER  (в процентах)    
    annot = path_matrix[-1][-1]
    counts = Counter(annot)
    
    wer = (counts['I'] + counts['D'] + counts['S'])/(counts['D'] + counts['S'] + counts['C']) * 100
    
    annot = path_matrix[-1][-1]
    ref_ali = []
    rec_ali = []
    
    ref_w_iter = iter(reference_words)
    rec_w_iter = iter(recognized_words)
    
    for symbol in annot:
        if 'D' in symbol:
            ref_ali.append(next(ref_w_iter))
            rec_ali.append("***")
        elif 'I' in symbol:
            ref_ali.append("***")
            rec_ali.append(next(rec_w_iter))
        elif 'C' in symbol or 'S' in symbol:
            ref_ali.append(next(ref_w_iter))
            rec_ali.append(next(rec_w_iter))
    
    ali = [ref_ali, rec_ali, annot]
    
    per = 0
    if (counts['D_p'] + counts['S_p'] + counts['C_p']) > 0:
        per = (counts['I_p'] + counts['D_p'] + counts['S_p'])/(counts['D_p'] + counts['S_p'] + counts['C_p']) * 100
    
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    return {"wer" : wer,
            "per": per, 
            "ali": ali}


In [236]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [305]:
import itertools
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
    sawer=float('inf')
    calc = calculate_wer_with_alignment(" ".join(reference_text), " ".join(recognized_text))
    ali=calc['ali']
    correct = calc["cor"]
    deletion = calc["del"]
    insertion = calc['ins']
    substitution = calc['sub']
    
    unique_rec_speakers = list(set(recognized_speakers))
    unique_ref_speakers = list(set(reference_speakers))
    
    possible_mappings = list(set(itertools.product(unique_ref_speakers, unique_rec_speakers)))
    possible_mappings = [[i for i in possible_mappings if i[1] == sp_id] for sp_id in  unique_rec_speakers]
    possible_mappings = list(itertools.product(*possible_mappings))
    
    for mapping in possible_mappings:
        mapping = {m[0]:m[1] for m in mapping}
        mapped_speakers = [mapping[sp] if sp in mapping else sp for sp in reference_speakers]
        
        s_i = len([(t,p) for t,p in zip(recognized_speakers, mapped_speakers) if t!=p])
        new_sawer = (insertion + deletion + substitution + s_i)/(deletion + substitution + correct + s_i)
        if new_sawer<sawer:
            sawer = new_sawer
    sawer *= 100
    return {"sawer" : sawer, 
            "ali": ali}


In [307]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]
    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
#     assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
#             "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
